#### Importações

In [612]:
import pandas as pd
import matplotlib.pyplot as plt

# Configurações para exibição do DataFrame
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_colwidth', None) # Mostrar todo o conteúdo das células

#### Importando os dataframes que serão analisados

In [613]:
df_original = pd.read_csv("../dados/nova_plataforma.csv")
df_gan = pd.read_csv("../dados/registros_gan.csv")

##### Removendo as colunas de df_original que não estão na df_gan e df_regessao

In [614]:
colunas_para_remover = ['platform', 'recommended (1 partial; 2 complete)', 'impacts_of_option-partial']
df_original.drop(colunas_para_remover, axis=1, inplace=True)

#### Analisando os registros gerados pela gan

##### Concatenado as estatisticas obtidas do df_original e df_gan

In [615]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [616]:
original_x_gan.head(25)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,121.425000,22.307377,19.903507,99.000,99.000,107.500,1.020000e+02,1.160000e+02,1.160000e+02,142.500000,141.000000,1.560000e+02,1.550000e+02
weight (t),18815.714286,18930.742200,5472.264453,4534.041928,10054.000,10068.000,16530.500,1.527250e+04,1.858400e+04,1.924000e+04,21182.000000,22777.000000,2.764700e+04,2.756400e+04
installation_date,38.857143,37.061800,5.639993,5.637194,31.000,31.000,34.500,3.100000e+01,4.100000e+01,3.600000e+01,42.500000,43.000000,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.381400,0.786796,0.712169,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,1.000000,3.000000e+00,3.000000e+00
number_of_legs,7.428571,7.047200,1.511858,1.704014,4.000,4.000,8.000,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.000000e+00,8.000000e+00
number_of_piles,17.142857,15.080300,10.106575,9.377763,5.000,5.000,8.500,5.000000e+00,2.000000e+01,1.300000e+01,23.000000,24.000000,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,138.355899,18.993420,18.998983,114.000,114.000,122.750,1.183850e+02,1.360000e+02,1.375415e+02,150.500000,157.981250,1.660000e+02,1.659990e+02
distance_to_coast (km),218.571429,225.556900,66.271807,65.445207,120.000,120.000,180.000,1.500000e+02,2.400000e+02,2.680000e+02,264.000000,281.000000,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.474800,0.534522,0.499390,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000006,0.000009,0.000009,0.000,0.000,0.000,1.850724e-11,7.800000e-08,2.096219e-08,0.000012,0.000011,2.300000e-05,2.300000e-05


#### Removendo registros que contém células que ultrapassam o intervalo da coluna (min e max)

In [617]:
def ultrapassou_intervalo(registro, coluna, df_original):
    if registro > df_original[coluna].max() or registro < df_original[coluna].min():
        return True
    else:
        return False

In [618]:
def remover_registros_fora_do_intervalo(df_gan, df_original):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            if ultrapassou_intervalo(linha[coluna], coluna, df_original):
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro
    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [619]:
df_gan_limpo = remover_registros_fora_do_intervalo(df_gan, df_original)

0


In [620]:
df_gan_limpo.shape

(10000, 25)

#### Removendo outliers com base na amplitude interquartil

In [621]:
def remover_outliers_iqr(df_gan):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            q1 = df_gan[coluna].quantile(0.25)
            q3 = df_gan[coluna].quantile(0.75)
            
            iqr = q3 - q1 # Amplitude interquantil
            limite_superior = q3 + 1.5 * iqr
            limite_inferior = q1 - 1.5 * iqr

            if linha[coluna] < limite_inferior or linha[coluna] > limite_superior:
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro

    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [636]:
#df_gan_limpo = remover_outliers_iqr(df_gan)

7017


In [623]:
df_gan.shape, df_gan_limpo.shape

((10000, 25), (10000, 25))

In [624]:
#df_gan_limpo.duplicated().sum()

#### Comparando novamente as bases de dados

In [625]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan_limpo.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [626]:
original_x_gan.head(26)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,121.425000,22.307377,19.903507,99.000,99.000,107.500,1.020000e+02,1.160000e+02,1.160000e+02,142.500000,141.000000,1.560000e+02,1.550000e+02
weight (t),18815.714286,18930.742200,5472.264453,4534.041928,10054.000,10068.000,16530.500,1.527250e+04,1.858400e+04,1.924000e+04,21182.000000,22777.000000,2.764700e+04,2.756400e+04
installation_date,38.857143,37.061800,5.639993,5.637194,31.000,31.000,34.500,3.100000e+01,4.100000e+01,3.600000e+01,42.500000,43.000000,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.381400,0.786796,0.712169,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,1.000000,3.000000e+00,3.000000e+00
number_of_legs,7.428571,7.047200,1.511858,1.704014,4.000,4.000,8.000,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.000000e+00,8.000000e+00
number_of_piles,17.142857,15.080300,10.106575,9.377763,5.000,5.000,8.500,5.000000e+00,2.000000e+01,1.300000e+01,23.000000,24.000000,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,138.355899,18.993420,18.998983,114.000,114.000,122.750,1.183850e+02,1.360000e+02,1.375415e+02,150.500000,157.981250,1.660000e+02,1.659990e+02
distance_to_coast (km),218.571429,225.556900,66.271807,65.445207,120.000,120.000,180.000,1.500000e+02,2.400000e+02,2.680000e+02,264.000000,281.000000,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.474800,0.534522,0.499390,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000006,0.000009,0.000009,0.000,0.000,0.000,1.850724e-11,7.800000e-08,2.096219e-08,0.000012,0.000011,2.300000e-05,2.300000e-05


In [627]:
describe_diferenca = (df_original.describe() - df_gan.describe()).transpose()
describe_diferenca.drop(columns='count', inplace=True)
describe_diferenca

,mean,std,min,25%,50%,75%,max
water_depth (m),3.003571,2.403870e+00,0.0,5.500000e+00,0.000000e+00,1.500000,1.000
weight (t),-115.027914,9.382225e+02,-14.0,1.258000e+03,-6.560000e+02,-1595.000000,83.000
installation_date,1.795343,2.799036e-03,0.0,3.500000e+00,5.000000e+00,-0.500000,0.000
type_of_production (1 oil and gas; 2 oil; 3 gas),0.190029,7.462674e-02,0.0,0.000000e+00,0.000000e+00,1.000000,0.000
number_of_legs,0.381371,-1.921558e-01,0.0,0.000000e+00,0.000000e+00,0.000000,0.000
number_of_piles,2.062557,7.288116e-01,0.0,3.500000e+00,7.000000e+00,-1.000000,1.000
height_of_jacket_or_sub-structure (m),-0.855899,-5.563548e-03,0.0,4.365000e+00,-1.541500e+00,-7.481250,0.001
distance_to_coast (km),-6.985471,8.266002e-01,0.0,3.000000e+01,-2.800000e+01,-17.000000,0.000
risk_to_other_users-complete,-0.046229,3.513296e-02,0.0,0.000000e+00,0.000000e+00,0.000000,0.000
risk_to_other_users-partial,0.000001,2.336334e-07,0.0,-1.850724e-11,5.703781e-08,0.000001,0.000


#### Analisando a semelhança dos dataframe com base no KSComplement e CorrelationSimilarity

In [628]:
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

In [629]:
df_original.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,99,22000,36,1,8,8,114.0,269,1,0.000000e+00,0.290,0.12,1110100,817000,87000,58500,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
1,112,20364,41,1,8,9,123.0,259,1,0.000000e+00,0.320,0.16,1180500,895500,92000,64000,0.00,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
2,141,15561,46,2,8,26,147.0,120,0,2.300000e-05,0.025,0.01,297654,530148,24277,31064,0.66,0.25,1.00,1.00,0.94,1.0,1.0,0.53,1.00
3,156,27647,44,2,8,32,166.0,240,0,1.500000e-05,0.040,0.02,487750,570818,40416,45266,1.00,0.50,1.00,1.00,0.66,1.0,1.0,1.00,0.57
4,103,18584,33,1,8,20,122.5,230,0,7.800000e-08,0.090,0.06,733082,511765,59588,41170,0.00,0.39,0.15,0.00,0.00,0.0,0.0,1.00,0.73


In [630]:
df_gan_limpo.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,139,10958,31,1,4,5,163.489,281,1,9.527802e-11,0.045,0.079,308628,433525,25873,31137,0.000,0.000,1.000,1.000,0.935,0.360,0.738,1.000,0.423
1,109,19042,41,1,8,8,119.623,239,1,1.092476e-08,0.292,0.142,975130,856351,90750,60643,0.000,0.000,0.997,0.811,0.755,0.310,0.271,0.998,0.685
2,102,22252,41,1,8,6,115.044,187,1,6.998479e-10,0.311,0.145,1113795,888571,91753,63192,0.000,0.000,0.999,0.327,0.670,0.367,0.313,1.000,0.700
3,109,16897,32,1,8,15,154.108,128,0,2.229369e-08,0.127,0.059,903838,593055,87102,37814,0.000,0.497,0.158,0.000,0.001,0.007,0.001,0.993,0.842
4,150,19605,45,3,8,29,165.684,120,0,2.292465e-05,0.029,0.019,335509,603480,50477,32979,0.019,0.500,0.999,0.085,0.128,0.235,0.261,0.538,0.903


In [631]:
# Supondo que A e B são seus DataFrames reais e sintéticos
A = df_original.copy()
B = df_gan.copy()

# Criar o metadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=A)
metadata_dict = metadata.to_dict()

# Gerar o relatório de qualidade
report = QualityReport()
report.generate(real_data=A, synthetic_data=B, metadata=metadata_dict)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 25/25 [00:00<00:00, 680.96it/s]|
Column Shapes Score: 77.54%



(2/2) Evaluating Column Pair Trends: |██████████| 300/300 [00:00<00:00, 441.23it/s]|
Column Pair Trends Score: 92.96%

Overall Score (Average): 85.25%



In [632]:
report.get_score()

0.8524825924895131

In [633]:
report.get_properties()

,Property,Score
0,Column Shapes,0.775392
1,Column Pair Trends,0.929573


In [634]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,installation_date,KSComplement,0.778571
1,type_of_production (1 oil and gas; 2 oil; 3 gas),KSComplement,0.817229
2,number_of_legs,KSComplement,0.904657
3,number_of_piles,KSComplement,0.781071
4,height_of_jacket_or_sub-structure (m),KSComplement,0.818157
5,risk_to_other_users-complete,KSComplement,0.953771
6,risk_to_other_users-partial,KSComplement,0.719929
7,risk_to_personnel-complete,KSComplement,0.756214
8,risk_to_personnel-partial,KSComplement,0.743414
9,impacts_of_option-complete,KSComplement,0.835214


In [635]:
report.get_details(property_name='Column Pair Trends')

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),CorrelationSimilarity,0.947123,0.622398,0.516644
1,installation_date,number_of_legs,CorrelationSimilarity,0.915343,0.614305,0.444990
2,installation_date,number_of_piles,CorrelationSimilarity,0.962265,0.681692,0.606223
3,installation_date,height_of_jacket_or_sub-structure (m),CorrelationSimilarity,0.818403,0.599779,0.236585
4,installation_date,risk_to_other_users-complete,CorrelationSimilarity,0.997098,-0.473868,-0.468064
5,installation_date,risk_to_other_users-partial,CorrelationSimilarity,0.987281,0.826764,0.801326
6,installation_date,risk_to_personnel-complete,CorrelationSimilarity,0.893652,-0.277817,-0.065121
7,installation_date,risk_to_personnel-partial,CorrelationSimilarity,0.953626,-0.274356,-0.181608
8,installation_date,impacts_of_option-complete,CorrelationSimilarity,0.977925,0.698996,0.654845
9,installation_date,technical_feasibility_or_challenge-complete,CorrelationSimilarity,0.893326,0.397458,0.184110
